In [1]:
import os
import faiss
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings

In [2]:
docs = SimpleDirectoryReader(input_dir="./data").load_data()

In [ ]:
docs[0].__dict__

In [3]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# ตั้งค่าให้แยกเอกสารเป็น chunk ย่อย ๆ
text_splitter = SentenceSplitter(
    separator=" ",       # ใช้เว้นวรรคเป็นตัวแยก
    chunk_size=1024,     # ขนาดของแต่ละ chunk (จำนวนตัวอักษร)
    chunk_overlap=128    # ความซ้อนของข้อความระหว่างแต่ละ chunk
)

# สร้าง ingestion pipeline ที่ประกอบด้วย text_splitter
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
    ]
)

# รัน pipeline กับเอกสารที่โหลดไว้ เพื่อแยกเป็น chunks
nodes = pipeline.run(
    documents=docs,
    in_place=True,       # แก้ไขเอกสารเดิมแทนที่จะสร้างใหม่
    show_progress=True,  # แสดง progress bar ขณะรัน
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:


# dimensions of BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatIP(d)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = 'BAAI/bge-m3'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cuda:0')

In [13]:
index = VectorStoreIndex(
    nodes=nodes,  embed_model=embed_model, show_progress=True,  # แสดง progress bar ขณะรัน
)

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
index = VectorStoreIndex.from_documents(docs, embed_model=embed_model)


In [14]:
retriever = index.as_retriever(similarity_top_k=5)
response = retriever.retrieve('LlamaIndex ')
for i in range(5):
    print('Response:', response[i].text,'\n')
    print('Score:', response[i].score)
    print('-'*50,'\n\n')

Response: LlamaIndex เป็นเครื่องมือที่ช่วยเชื่อมโยงโมเดลภาษาเข้ากับฐานข้อมูลหรือเอกสารได้อย่างง่ายดาย จุดเด่นคือรองรับการสกัด metadata เช่น ชื่อเรื่อง คำสำคัญ สาระสังเขป ฯลฯ เพื่อช่วยให้การค้นหาข้อมูลมีประสิทธิภาพมากขึ้น

โมเดล embedding ที่นิยมใช้กับภาษาไทยในปัจจุบัน เช่น BAAI/bge-m3 สามารถนำมาใช้ร่วมกับ LlamaIndex เพื่อสร้างระบบ RAG ที่รองรับภาษาไทยได้ดี 

Score: 0.6144973488022796
-------------------------------------------------- 


Response: ปัญญาประดิษฐ์ (AI) ในประเทศไทยกำลังพัฒนาอย่างรวดเร็ว หน่วยงานทั้งภาครัฐและเอกชนให้ความสำคัญกับการนำ AI ไปประยุกต์ใช้ในหลายด้าน เช่น การแพทย์ การศึกษา การเกษตร และอุตสาหกรรม

หนึ่งในหัวข้อที่ได้รับความสนใจมากคือ การนำโมเดลภาษา (LLM) มาช่วยในการวิเคราะห์และสรุปข้อมูลภาษาไทย เพื่อเพิ่มประสิทธิภาพการทำงานและลดเวลาที่ใช้ในการประมวลผลข้อมูล 

Score: 0.4605406966344382
-------------------------------------------------- 


Response: ระบบ Retrieval-Augmented Generation (RAG) เป็นเทคนิคที่ช่วยให้โมเดลภาษาใหญ่สามารถตอบคำถามจากข้อมูลที่อยู่นอกเหนือจากที่ใ

IndexError: list index out of range

In [ ]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

In [ ]:
from llama_index.core import (

    load_index_from_storage,
    StorageContext,
)

from IPython.display import Markdown, display
from llama_index.vector_stores.faiss import FaissVectorStore

In [ ]:
persist_dir="./storage/"
vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)

In [ ]:
retriever = index.as_retriever(similarity_top_k=5)
response = retriever.retrieve('AI สำคัญกับประเทศไทยอย่างไร')
for i in range(5):
    print('Response:', response[i].text,'\n')
    print('Score:', response[i].score)
    print('-'*50,'\n\n')